![Nuclio logo](https://nuclio.school/wp-content/uploads/2018/12/nucleoDS-newBlack.png)

# TFM - Impacto de la alimentación sobre la salud de las personas

## Tabla de Contenidos

* [A. Introduccion](#introduction)
* [B. Importar Librerias](#libraries)
* [C. Importar datos](#data)

## A. Introducción <a class="anchor" id="introduction"></a>

## B. Importar Librerías<a class="anchor" id="libraries"></a>

In [345]:
import os
import sys
import numpy as np
import pandas as pd
from datetime import datetime

from matplotlib import pyplot as plt
import plotly.express as px


import seaborn as sns
from sklearn import preprocessing

import pickle


## C. Importar datos<a class="anchor" id="data"></a>

### Datos consumos por CCAA

In [317]:
PATH_COMUNIDADES = 'data/comunidades/'

In [318]:
# Leemos los nombres de los ficheros
paths_comunidades = [path for path in os.listdir(PATH_COMUNIDADES) if path.endswith('.xlsx')]

# Creamos una lista para ir añadiendo los dataframes leidos
dfs = list()

for path_comunidad in paths_comunidades:
    # Extraemos el nombre de la comunidad
    nombre_comunidad = os.path.splitext(path_comunidad)[0]

    # Leemos el fichero, reseteamos el index y le cambiamos el nombre de la columna a ID
    fichero = pd.read_excel('./data/comunidades/'+path_comunidad).reset_index(drop=True).rename(columns={'Unnamed: 0': 'ID'})

    # Descartamos las filas donde el la columna BIO no es nula y asignamos el nombre de la comunidad a una nueva columna.
    fichero_sin_bio = fichero[fichero['SEGMENTO BIO'].isna()].reset_index(drop=True).assign(COMUNIDAD=nombre_comunidad)

    # Añadimos el fichero leido a nuestra lista de dataframes
    dfs.append(fichero_sin_bio)

# Montamos el dataframe final
df_comunidades = pd.concat(dfs, ignore_index=True)

# Eliminamos la columna SEGMENTO BIO
df_comunidades.drop('SEGMENTO BIO', axis=1, inplace=True)

# Cambiamos el orden de la columna COMUNIDAD
cols = df_comunidades.columns.tolist()
cols = cols[:5] + cols[-1:] + cols[5:-1]
df_comunidades = df_comunidades[cols]

In [319]:
df_comunidades.head(20)

,ID,SECTOR,SECCION,CATEGORIA,FAMILIA,COMUNIDAD,SEMANA 40-16,SEMANA 41-16,SEMANA 42-16,SEMANA 43-16,...,SEMANA 34-19,SEMANA 35-19,SEMANA 36-19,SEMANA 37-19,SEMANA 38-19,SEMANA 39-19,SEMANA 40-19,SEMANA 41-19,SEMANA 42-19,SEMANA 43-19
0,TOTAL PGC,NaN,NaN,NaN,NaN,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,2652.71,2853.52,2699.68,2703.08,2484.19,2853.67,2844.06,2412.15,2559.18,2647.53
1,ALIM. Y BEBIDAS,ALIM. Y BEBIDAS,NaN,NaN,NaN,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,1328.36,1421.87,1364.16,1405.32,1269.16,1457.10,1477.37,1288.87,1375.99,1442.49
2,ALIM.SECA,ALIM. Y BEBIDAS,ALIM.SECA,NaN,NaN,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,704.00,782.74,760.74,791.09,717.49,829.03,829.07,739.01,813.17,891.25
3,ACEITE,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,NaN,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,60.80,72.97,67.83,71.55,62.32,73.07,76.68,61.45,60.28,63.69
4,GIRASOL..,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,GIRASOL..,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,18.18,20.78,20.41,20.16,18.68,21.06,22.47,16.94,17.49,19.44
5,OLIVA,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,OLIVA,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,39.07,48.58,44.35,47.56,40.15,47.73,50.83,41.63,39.63,41.40
6,RESTO ACEITES,ALIM. Y BEBIDAS,ALIM.SECA,ACEITE,RESTO ACEITES,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,3.54,3.61,3.06,3.84,3.49,4.29,3.39,2.88,3.17,2.85
7,ADITIVOS COCINA,ALIM. Y BEBIDAS,ALIM.SECA,ADITIVOS COCINA,NaN,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,13.87,15.52,14.98,16.48,15.77,16.12,16.09,14.38,14.43,17.00
8,ADEREZOS,ALIM. Y BEBIDAS,ALIM.SECA,ADITIVOS COCINA,ADEREZOS,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,0.28,0.25,0.25,0.27,0.22,0.31,0.24,0.19,0.18,0.21
9,ESPECIAS,ALIM. Y BEBIDAS,ALIM.SECA,ADITIVOS COCINA,ESPECIAS,Ceuta_y_Melilla,0.0,0.0,0.0,0.0,...,5.53,6.47,6.14,6.45,6.15,6.57,6.62,6.22,6.01,8.11


### Datos población por CCAA

In [329]:
PATH_POBLACION = './data/poblacion/'

In [454]:
df_poblacion = pd.read_excel(PATH_POBLACION+'poblacion_comunidades.xlsx', skiprows=1).set_index('CC.AA.')

In [455]:
df_poblacion

,2016,2017,2018,2019,2020
CC.AA.,,,,,
Ceuta_y_Melilla,170545,171079,171528,171264,171278
La_Rioja,315794,315381,315675,316798,313914
Pais_Vasco,2189534,2194158,2199088,2207776,2220504
Navarra,640647,643234,647554,654214,661197
Murcia,1464847,1470273,1478509,1493898,1511251
Madrid,6466996,6507184,6578079,6663394,6779888
Galicia,2718525,2708339,2701743,2699499,2701819
Extremadura,1087778,1079920,1072863,1067710,1063987
C_Valenciana,4959968,4941509,4963703,5003769,5057353


### Datos diabetes por CCAA

In [326]:
PATH_ENFERMEDADES = './data/enfermedades/'

In [327]:
df_diabetes = pd.read_excel(PATH_ENFERMEDADES + 'diabetes.xlsx', skiprows=1)

In [328]:
df_diabetes

,CC.AA.,2016,2017,2018,2019,2020
0,Ceuta_y_Melilla,50.36,35.33,27.14,20.33,47.11
1,La_Rioja,15.88,15.51,19.18,15.20,15.82
2,Pais_Vasco,15.19,13.03,15.14,15.07,16.65
3,Navarra,20.41,20.21,16.13,15.77,16.34
4,Murcia,20.92,25.17,21.24,22.40,30.91
5,Madrid,10.90,10.49,10.23,8.28,10.03
6,Galicia,16.32,14.40,12.66,11.08,12.88
7,Extremadura,19.58,17.68,17.88,16.88,22.17
8,C_Valenciana,19.39,20.06,25.27,25.50,28.49
9,Catalunya,19.32,20.81,19.91,18.47,21.28


# 1. Análisis exploratorio

## 1.1. Análisis datos consumo

In [234]:
# Sacamos la lista de todos los valores de cada sector, sección, categoría y familia de alimentos
lista_sectores = list(df_comunidades[~df_comunidades['SECTOR'].isna()]['SECTOR'].unique())
lista_secciones = list(df_comunidades[~df_comunidades['SECCION'].isna()]['SECCION'].unique())
lista_categorias = list(df_comunidades[~df_comunidades['CATEGORIA'].isna()]['CATEGORIA'].unique())
lista_familias = list(df_comunidades[~df_comunidades['FAMILIA'].isna()]['FAMILIA'].unique())

In [229]:
len(lista_sectores), len(lista_secciones), len(lista_categorias), len(lista_familias)

(3, 11, 95, 414)

In [241]:
# Copiamos esta lista para modificarla
lista_sectores_diabetes = lista_sectores.copy()

# Eliminamos los sectores que no están relacionados con la diabetes
lista_sectores_diabetes.remove('DROGUERIA Y PERFUMERIA')
lista_sectores_diabetes

['ALIM. Y BEBIDAS', 'PROD.FRESCOS']

In [254]:
# Secciones dentro de lista_sectores_diabetes
lista_secciones_sectores_diabetes = list(df_comunidades.loc[df_comunidades['SECTOR'].isin(lista_sectores_diabetes), 'SECCION'].unique())[1:]

# Copiamos la lista de los secciones de los sectores relacionados con la diabetes
lista_secciones_diabetes = lista_secciones_sectores_diabetes.copy()

# Creamos una lista de las secciones a eliminar
secciones_sin_diabetes = ['CONSERVAS', 'CHARCUTERIA', 'PL. COCINADOS Y PRECOCINADOS', 'QUESOS']

for i in range(len(secciones_sin_diabetes)):
    lista_secciones_diabetes.remove(secciones_sin_diabetes[i])

# Lista de las secciones relacionadas con la diabetes
lista_secciones_diabetes

['ALIM.SECA', 'BEBIDAS', 'LECHE Y BATIDOS', 'CONGELADOS', 'DERV.LACTEOS']

In [307]:
# Categorias de las secciones en lista_secciones_diabetes
lista_categorias_secciones_diabetes = list(df_comunidades.loc[df_comunidades['SECCION'].isin(lista_secciones_diabetes), 'CATEGORIA'].unique())[1:]

# Copiamos la lista de los categorias de las secciones relacionados con la diabetes
lista_categorias_diabetes = lista_categorias_secciones_diabetes.copy()

# Creamos una lista de las categorias a eliminar
categorias_sin_diabetes = [
    'ACEITE', 'ADITIVOS COCINA', 'ALIMENTOS INFANTILES', 'ALIMENTOS MASCOTAS', 'APERITIVOS', 'ARROZ', 'CAFES', 'DIETETICOS', 'FRUTOS SECOS',
    'INFUSIONES', 'LEGUMBRES SECAS', 'PANADERIA INDUSTRIAL', 'PASTAS', 'SOPAS Y DESHIDRATADOS', 'AGUAS', 'CERVEZA', 'ESPUMOSOS', 'VINOS',
    'LECHE LIQUIDA Y BEBIDAS VEGETALES', 'PESCADO CONGELADO', 'PESCADO PREPARADO CONGELADO', 'PLATOS PREPARADOS CONGELADOS', 'VERDURAS CONGELADAS',
    'MANTEQUILLA', 'MARGARINA', 'QUESO TIPO PETIT'
]

for i in range(len(categorias_sin_diabetes)):
    lista_categorias_diabetes.remove(categorias_sin_diabetes[i])

# Lista de las categorias relacionadas con la diabetes
lista_categorias_diabetes

['AZUCAR Y EDULCORANTES',
 'BOLLERIA INDUSTRIAL',
 'CACAO',
 'CEREALES DESAYUNO',
 'CHOCOLATES',
 'DULCES NAVIDENOS',
 'GALLETAS',
 'GOLOSINAS',
 'REPOSTERIA',
 'SALSAS',
 'BEBIDAS ALCOHOLICAS',
 'BEBIDAS REFRESCANTES',
 'ZUMOS',
 'BATIDOS',
 'HORCHATA.',
 'LECHE NO LIQUIDA',
 'HELADOS',
 'OTROS CONGELADOS',
 'NATA',
 'POSTRES PREPARADOS',
 'YOGURES']

In [316]:
# Familias de alimentos de las categorias en lista_categorias_diabetes
lista_familias_categorias_diabetes = list(df_comunidades.loc[df_comunidades['CATEGORIA'].isin(lista_categorias_diabetes), 'FAMILIA'].unique())[1:]

# Copiamos la lista de las familias de alimentos de las categorias relacionados con la diabetes
lista_familias_diabetes = lista_familias_categorias_diabetes.copy()

# Creamos una lista de las categorias a eliminar
famlias_sin_diabetes = [
    'EDULCORANTE', 'LEVADURA', 'L.EVAPORADA', 'L.POLVO', 'BASES PIZZA', 'CHURROS', 'FRUTAS', 'HIELO', 'HOJALDRES MASAS RELLENAS', 'HOJALDRES SIN RELLENAR',
    'NATA', 'YOGUR NATURAL'
]

for i in range(len(famlias_sin_diabetes)):
    lista_familias_diabetes.remove(famlias_sin_diabetes[i])

# Lista de las categorias relacionadas con la diabetes
len(lista_familias_diabetes)

120

## 1.2. Dataset familias de alimentos relacionados con la diabetes

In [396]:
# Creamos un dataset de las familias de alimentos que tienen relación con la diabetes
df_diabetes = df_comunidades[df_comunidades['FAMILIA'].isin(lista_familias_diabetes)].groupby('COMUNIDAD').sum().T

# Editamos el índice para que tenga un formato adecuado para ser formateado a fecha (quitamos 'SEMANA', 1 para que la semana empiece en lunes, la semana y el año)
df_diabetes.index = '1 ' + df_diabetes.index.str[-5:-3] + ' ' + df_diabetes.index.str[-2:]
df_diabetes.index = pd.to_datetime(df_diabetes.index, format="%w %W %y")

# Cambiamos la granularidad a mensual
df_diabetes = df_diabetes.groupby(pd.Grouper(freq='M')).sum()

In [406]:
fig = px.line(df_diabetes, title='Total de consumo de alimentos relacionados con la diabetes')
fig.show()

## 1.3. Normalizar datos con la población de cada comunidad

In [739]:
# Dividimos el DF para cada año
df_diabetes_2017 = df_diabetes['2017':'2017']
df_diabetes_2018 = df_diabetes['2018':'2018']
df_diabetes_2019 = df_diabetes['2019':'2019']

In [746]:
# Usamos la función apply para dividir los datos por la población de cada comunidad. Multiplicamos por 100 para que nos den mejores valores
df_diabetes_2017_normalizado = df_diabetes_2017.apply(lambda x: x / df_poblacion['2017'][x.name] * 100)
df_diabetes_2018_normalizado = df_diabetes_2018.apply(lambda x: x / df_poblacion['2018'][x.name] * 100)
df_diabetes_2019_normalizado = df_diabetes_2019.apply(lambda x: x / df_poblacion['2019'][x.name] * 100)

In [756]:
# Unimos el DF con los valores ya normalizados
df_diabetes_normalizado = pd.concat([df_diabetes_2017_normalizado, df_diabetes_2018_normalizado, df_diabetes_2019_normalizado])

In [764]:
fig = px.line(
    df_diabetes_normalizado,
    title='Consumo normalizado de alimentos relacionados con la diabetes por CCAA',
    labels = {
        'index' : 'Tiempo',
        'value' : 'Consumo / Población * 100'
    }
)
fig.show()